## This notebook generates a rxml file for Solid State reactions in the MgFeSiO4 Stixrude solution phases

In [ ]:
from thermocodegen.codegen import reaction
import sympy as sym
import os
from glob import glob
sym.init_printing()

Set a reference string for this Notebook

In [ ]:
reference = 'Thermocodegen-v0.6/share/thermocodegen/examples/Systems/MgFeSiO2_stixrude/notebooks/Generate_reactions.ipynb'

List of phases present in this set of reactions

In [ ]:
phase_names = ['Olivine', 'Wadsleyite', 'Ringwoodite']

Path to the thermodynamic database tarball file that this set of reactions are built with

Here we will use a pre-compiled Thermodynamic database available as a DOI download from zenodo at [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.6350797.svg)](https://doi.org/10.5281/zenodo.6350797)

In [ ]:
db = 'https://zenodo.org/record/6350797/files/MgFeSi2O4_Stixrude.tar.gz'

Instantiate a Reaction object. This is initialized using the 'from_database' class method. Initializing in this way requires the total number of reaction (total_reactions), a list of phase names (phase_names) and a path to a thermocodegen generated thermodynamic database.

In [ ]:
rxn = reaction.Reaction.from_database(total_reactions=4, phase_names=phase_names, database=db)

The 'model_dict' attribute of this object contains the current state of the information that we have given and what has been extracted from the thermodynamic database 'fo_fa_coder_db.tar.gz'

In [ ]:
rxn.model_dict

#### Set some utility indices for referencing phases and endmembers (this should be automated somehow)

In [ ]:
iOl = 0
iWa = 1
iRi = 2
kMg = 0
kFe = 1

### Define the reactions in this system, and then write SymPy that describes the reactions rates

First we pull out the SymPy symbols for the independent variables (temperature T, pressure P, concentration C and phase fraction Phi). Concentration C is a sym.MatrixSymbol of dimension N (number of phases) by Kmax (maximum number of endmembers). Phase fraction Phi is a sym.MatrixSymbol of dimension N (number of phases) by 1 (i.e., a vector).

In [ ]:
# Variables
T = rxn.T
P = rxn.P
C = rxn.C
Phi = rxn.Phi

Now we pull out the special symbol reserved for the affinity, A. This is a sym.MatrixSymbol of dimension J (number of reactions) by 1.

In [ ]:
# Affinity
A = rxn.A

Now we define the parameters used in the reaction rate expressions. These should be sym.Symbol objects. We also need to create lists that contain the parameter names as strings, their units (in string form) and the corresponding SymPy symbols. Note that these need to be ordered correctly.

In [ ]:
# Parameters
T0 = sym.Symbol('T0')
R = sym.Symbol('R')

params = ['T0','R']
units = ['K','J/(mol K)']
symparam = [T0, R]

### Forsterite to Wadleyite Reaction

Set up the forsterite to wadsleyite reaction. This requires a list of tuples for both the reactants and products. Each tuple consists of strings of 

(name,phase,endmember)

where 
    **name**:  is an arbitrary name describing the reactant endmenber,
    **phase**: is the phase containing the endmember
    **endmember**: the name of the endmember that is reacting in that phase
    
The phase and endmember names should be consistent with whatever they are called in the thermodynamic database.

In [ ]:
# reaction 0
j = 0

# Reactants
Fo_Ol = ('Fo_Ol','Olivine','Forsterite_stixrude')

# Products
MgWa_Wa = ('MgWa_Wa','Wadsleyite','MgWadsleyite_stixrude')

Now we write down the SymPy expression for the reaction. This, along with the list of reactants, products, parameters, units and variable symbols are passed into the 'add_reaction_to_model' function. This function also requires a 'name' field, which should be consistent with the variable assigned to the SymPy expression; for example, here our expression is Fo_melting = ..., so our name is 'Fo_melting'.

In [ ]:

# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S0p = Phi[iOl]
S0m = Phi[iWa]
Fo_MgWa = sym.Piecewise((rp*S0p*A[j]/R/T,A[j]>=0),(rm*S0m*A[j]/R/T,A[j]<0),(0,True))
Fo_MgWa

In [ ]:
reactants = [ Fo_Ol]
products = [ MgWa_Wa]
rxn.add_reaction_to_model('Fo_MgWa', reactants, products, Fo_MgWa, list(zip(params,units,symparam)))

### Repeat for Fe endmember

In [ ]:
# reaction 1
j = 1

# Reactants
Fa_Ol = ('Fa_Ol','Olivine','Fayalite_stixrude')

# Products
FeWa_Wa = ('FeWa_Wa','Wadsleyite','FeWadsleyite_stixrude')

In [ ]:
# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S0p = Phi[iOl]
S0m = Phi[iWa]
Fa_FeWa = sym.Piecewise((rp*S0p*A[j]/R/T,A[j]>=0),(rm*S0m*A[j]/R/T,A[j]<0),(0,True))
Fa_FeWa

In [ ]:
reactants = [ Fa_Ol]
products = [ FeWa_Wa]
rxn.add_reaction_to_model('Fa_FeWa', reactants, products, Fa_FeWa, list(zip(params,units,symparam)))

### Do the same for the Wad to Ringwoodite reactions

In [ ]:
# reaction 2
j = 2

# Reactants
MgWa_Wa = ('MgWa_Wa','Wadsleyite','MgWadsleyite_stixrude')

# Products
MgRi_Ri = ('MgRi_Ri','Ringwoodite','MgRingwoodite_stixrude')

In [ ]:
# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S1p = Phi[iWa]
S1m = Phi[iRi]
MgWa_MgRi = sym.Piecewise((rp*S1p*A[j]/R/T,A[j]>=0),(rm*S1m*A[j]/R/T,A[j]<0),(0,True))
MgWa_MgRi

In [ ]:
reactants = [ MgWa_Wa ] 
products = [ MgRi_Ri ]
rxn.add_reaction_to_model('MgWa_MgRi', reactants, products, MgWa_MgRi, list(zip(params,units,symparam)))

### Repeat for Fe endmembers

In [ ]:
# reaction 3
j = 3

# Reactants
FeWa_Wa = ('FeWa_Wa','Wadsleyite','FeWadsleyite_stixrude')

# Products
FeRi_Ri = ('FeRi_Ri','Ringwoodite','FeRingwoodite_stixrude')

In [ ]:
# SymPy expression for reaction rate
r = sym.exp(-T0/T)
rp = r
rm = r

S1p = Phi[iWa]
S1m = Phi[iRi]
FeWa_FeRi = sym.Piecewise((rp*S1p*A[j]/R/T,A[j]>=0),(rm*S1m*A[j]/R/T,A[j]<0),(0,True))
FeWa_FeRi

In [ ]:
reactants = [ FeWa_Wa ] 
products = [ FeRi_Ri ]
rxn.add_reaction_to_model('FeWa_FeRi', reactants, products, MgWa_MgRi, list(zip(params,units,symparam)))

### The model_dict has now been updated to contain all of the information for these reactions

In [ ]:
rxn.model_dict

Return a dictionary of settable name value pairs

In [ ]:
values_dict = rxn.get_values()
values_dict

Update some of these values...


In [ ]:
values_dict.update(dict(name='MgFeSiO4_stixrude',
                        reference=reference,
                        T0=2000.,
                        R=8.31442))
values_dict

... and update them in the model_dict using the 'set_values' function

In [ ]:
rxn.set_values(values_dict)
rxn.model_dict

### Generate Spud XML files

Set some directories

In [ ]:
HOME_DIR = os.path.abspath(os.curdir)
SPUD_DIR = HOME_DIR+'/../reactions'

try:
    os.mkdir(SPUD_DIR)
except:
    pass

### Dump the spud file

In [ ]:
rxn.to_xml(path=SPUD_DIR)